<a href="https://colab.research.google.com/github/Seyoung-Jung/Tobigs-HW/blob/master/week4/SVM/assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### TOBIG'S 14기 정규세션 4주차 SVM 
### ASSIGNMENT1. Multiclass SVM 구현

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


모델에 학습시킬 땐 train 데이터만 사용하므로 test 데이터와 split한 뒤에 scaling 해줘야 한다.

In [ ]:
from sklearn.model_selection import train_test_split #test/train 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

X_train 데이터에 fit하여 scaling한 후에, fit한 scaler를 X_test에 적용시켜주어야 한다.

In [ ]:
scal = StandardScaler() #scaling
X_train = scal.fit_transform(X_train)
X_test = scal.transform(X_test)

In [ ]:
y_train = pd.get_dummies(y_train) #one hot encoding

In [ ]:
# One VS Rest 구현을 위해 1) class가 0 or not 2)class가 1 or not을 구분하기 위한 머신 등을 미리 만들어주자
svm_1 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_2 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_3 = SVC(kernel ='rbf', C = 5, gamma = 5)

In [ ]:
svm_1.fit(X_train,y_train.iloc[:,0]) # 데이터 클레스가 0 or not 구분해주는 머신
svm_2.fit(X_train,y_train.iloc[:,1]) # 데이터 클레스가 1 or not 구분해주는 머신
svm_3.fit(X_train,y_train.iloc[:,2]) # 데이터 클레스가 2 or not 구분해주는 머신
print(svm_1.predict(X_test)) #데이터 클래스가 0 or not을 구분해주는 애를 통해서 테스트 데이터의 클래스가 0인지, 0이 아닌인지 예측해보자

print(svm_1.decision_function(X_test)) #decision_function hyperplane과의 거리를 구하는 방법(필요하다면 활용해주세요!)

[0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0]
[-1.12359969 -0.86782512 -0.65599247 -0.50194294 -0.76541147 -0.8819188
  1.07735938 -0.99156769  0.50201986 -0.9984315  -0.84532712  0.17062549
  0.34917127 -0.9813287  -0.72783399 -0.93313988  1.28153212 -0.56827872
 -0.73092732 -0.99670034  0.43553308 -0.96967771 -0.83939495 -1.03305682
 -0.75566609  1.13888006  0.42965012 -1.04268452 -0.93608147 -1.06090982]


In [ ]:
# 부호가 모든 같은 경우가 있는가? > 모두 동점인 경우가 생길 것
for i in range(len(X_test)):
    # ~. decision_function을 이용하면 해당 데이터가 하이퍼플레인으로부터 얼마나 떨어져있는지 '거리'가 나온다!
    # 다음은 그 값의 부호를 이용해 모두가 동점인 경우가 있는지 출력하는 함수 
    if (np.sign(svm_1.decision_function(X_test)[i]) == np.sign(svm_2.decision_function(X_test)[i])) and (np.sign(svm_2.decision_function(X_test)[i]) == 
    np.sign(svm_3.decision_function(X_test)[i])):
        print(i)

3
17
18


In [ ]:
## Case 1 : One vs Rest SVM을 이부분에 구현해주세요 위 결과들을 이용해서 multi class SVM을 직접 구현해주세요! 하드코딩이 하시기 편할겁니다.
y_pred = []
predict_1 = svm_1.predict(X_test)
predict_2 = svm_2.predict(X_test)
predict_3 = svm_3.predict(X_test)
distance_1 = svm_1.decision_function(X_test)
distance_2 = svm_2.decision_function(X_test)
distance_3 = svm_3.decision_function(X_test)

for i in range(len(X_test)):  # test 데이터 하나씩 예측한 결과를 y_pred에 담는다.
    setosa =predict_1[i]       # setosa이면 1, 아니면 0
    versicolor = predict_2[i]   # versicolor이면 1, 아니면 0
    virginica = predict_3[i]    # virginica이면 1, 아니면 0
    if (setosa + versicolor + virginica)==1:  # 셋 중에 하나만 맞다고 예측한 경우
        if setosa==1:y_pred.append('setosa')
        elif versicolor==1:y_pred.append('versicolor')
        else:y_pred.append('virginica')
    else:                                     # 동점과 같은 상황으로 인해 바로 하나의 클래스로 판단할 수 없는 경우
        distance = [np.abs(distance_1[i]), np.abs(distance_2[i]), np.abs(distance_3[i])]  # distance는 이 test 데이터의 각 초평면까지의 거리를 담는다.
        min_distance = distance.index(min(distance))                 # distance가 가장 짧은 클래스의 인덱스를 구해 예측 결과로 판단한다.
        if min_distance==0:y_pred.append('setosa')
        elif min_distance==1:y_pred.append('versicolor')
        else:y_pred.append('virginica')

from sklearn import metrics
print(y_pred)
print(metrics.accuracy_score(y_test,y_pred))

['versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'setosa', 'virginica', 'setosa', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'versicolor', 'versicolor', 'setosa', 'versicolor', 'virginica', 'virginica', 'setosa', 'virginica', 'versicolor', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'virginica', 'versicolor']
0.8666666666666667


In [ ]:
# Case 2 : One vs One SVM을 이 부분에 구현해주세요. (선택사항)
from collections import Counter

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)
scal = StandardScaler() #scaling
X_train = scal.fit_transform(X_train)
X_test = scal.transform(X_test)   # one-hot encoding을 하지 않은 원래 y_train 데이터를 사용하기 위해 split과 scaling을 다시 해준다.
X_train = (pd.DataFrame(X_train).reset_index()).drop(['index'],axis=1)  # split할 때 원래 데이터의 인덱스를 끌고 오므로 이를 없애기 위해 reset_index() 후 'index' 열을 제거한다.
y_train = (pd.DataFrame(y_train).reset_index()).drop(['index'],axis=1)

row01 = y_train[(y_train.species=='setosa') | (y_train.species=='versicolor')].index  # y가 두 클래스에 해당하는 행만(120행 중에 약 80행) 가져와야 학습이 가능하므로 row 인덱스를 저장한다.
row02 = y_train[(y_train.species=='setosa') | (y_train.species=='virginica')].index
row12 = y_train[(y_train.species=='versicolor') | (y_train.species=='virginica')].index

svm_01 = SVC(kernel ='rbf', C = 5, gamma = 5).fit(X_train.iloc[row01,:],y_train.iloc[row01])
svm_02 = SVC(kernel ='rbf', C = 5, gamma = 5).fit(X_train.iloc[row02,:],y_train.iloc[row02])
svm_12 = SVC(kernel ='rbf', C = 5, gamma = 5).fit(X_train.iloc[row12,:],y_train.iloc[row12])

y_pred = []
predict_01 = svm_01.predict(X_test)
predict_02 = svm_02.predict(X_test)
predict_12 = svm_12.predict(X_test)
for i in range(len(X_test)):
    compitition = [predict_01[i], predict_02[i], predict_12[i]]  # 세 가지 예측 결과를 저장한다. 1:1 모델이 3개라 동점이 나오는 경우는 고려할 필요가 없다.
    y_pred.append(Counter(compitition).most_common(1)[0][0])  # 가장 많이 승리한 클래스 최종 투표 결과를 y_pred에 넣어준다.

print(y_pred)
metrics.accuracy_score(y_test,y_pred)

['versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'setosa', 'virginica', 'setosa', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'versicolor', 'versicolor', 'setosa', 'versicolor', 'virginica', 'virginica', 'setosa', 'virginica', 'versicolor', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'virginica', 'versicolor']


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8666666666666667

In [ ]:
# 원래 라이브러리가 제공하는 multi class SVM과 여러분이 구현한 multiclass SVM 결과를 비교해주세요
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2, random_state=48)

svm_4 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_4.fit(X_train_2, y_train_2)
y_pred = svm_4.predict(X_test_2)

print(y_pred)
metrics.accuracy_score(y_test_2,y_pred)

['versicolor' 'versicolor' 'virginica' 'virginica' 'virginica' 'virginica'
 'setosa' 'virginica' 'setosa' 'versicolor' 'virginica' 'setosa' 'setosa'
 'virginica' 'versicolor' 'versicolor' 'setosa' 'versicolor' 'virginica'
 'virginica' 'setosa' 'virginica' 'virginica' 'versicolor' 'virginica'
 'setosa' 'setosa' 'virginica' 'virginica' 'versicolor']


0.8666666666666667

One vs Rest와 One vs One, 기존 라이브러리에서 제공하는 multi class SVM 모두 0.8667의 동일한 정확도를 보였다. 각각 하나씩 비교해보면 한 두개 다른 경우가 있지만 결과적으로 모두 동일한 예측 능력을 보이고 있다.